### Hand Tracking and Finger Counting System using OpenCV

In [13]:
# Imports 
import cv2
import numpy as np
from sklearn.metrics import pairwise  # type: ignore

In [14]:
background = None
accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

### Background Accumulation Function

In [15]:
def calc_accum_avg(frame, accumulated_weight, background):
    """
    Updates the accumulated background with the given frame using a specified weight.
    
    Parameters:
        frame (numpy.ndarray): The current frame to be accumulated.
        accumulated_weight (float): The weight of the current frame in the accumulation process. 
        background (numpy.ndarray): The current accumulated background to be updated.

    Returns:
        numpy.ndarray: The updated accumulated background.
    """
    if background is None:
        background = np.float32(frame)
    else:
        frame_float32 = np.float32(frame)
        cv2.accumulateWeighted(frame_float32, background, accumulated_weight)
    
    return background

### Hand Segmentation Function

In [16]:
def segment(frame, background, threshold_min=25):
    """
    Segments the largest region of change in an image based on the difference with a given background.
    
    Parameters:
        frame (numpy.array): The current image where segmentation is performed.
        background (numpy.array): The background image to compare against the current frame.
        threshold_min (int): The minimum threshold value for binarization.
    
    Returns:
        tuple: A tuple containing the binarized image and the contour of the largest segmented area, or None if no contours are found.
    """
    # Calculate the absolute difference between the background and the current frame.
    diff = cv2.absdiff(background.astype('uint8'), frame)
    
    # Apply thresholding to binarize the image.
    _, thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)
    
    # Find contours in the binarized image.
    contours, _ = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Return None if no contours are found.
    if not contours:
        return None
    
    # Find the contour with the maximum area and return it.
    hand_segment = max(contours, key=cv2.contourArea)
    return (thresholded, hand_segment)

### Count Fingers Function

In [17]:
def count_fingers(thresholded, hand_segment):
    """
    Counts the fingers visible in a given hand segment.
    
    Parameters:
        thresholded (numpy.array): The thresholded image of the hand.
        hand_segment (numpy.array): The contour of the hand segment.
    
    Returns:
        int: The number of fingers counted.
    """
    # Calculate the convex hull of the hand segment.
    conv_hull = cv2.convexHull(hand_segment)
    
    # Find the extreme points on the convex hull.
    top = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    # Calculate the center of the hand.
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    # Calculate the Euclidean distance between the center and the extreme points.
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    max_distance = distance.max()
    
    # Define the radius of the circular ROI.
    radius = int(0.9 * max_distance)
    
    # Create a circular ROI.
    circular_roi = np.zeros(thresholded.shape[:2], dtype='uint8')
    cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
    
    # Use the circular ROI to isolate the region of interest in the thresholded image.
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)
    
    # Find contours in the circular ROI.
    contours, _ = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        out_of_wrist = (cY + (cY * 0.25)) > (y + h)
        limit_points = ((2 * np.pi * radius * 0.25) > contour.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count

### Hand Tracking and Finger Counting Loop

In [ ]:
cam = cv2.VideoCapture(0) 

num_frames = 0
accumulated_weight = 0.5
background = None

roi_top = 20
roi_bottom = 300
roi_left = 300
roi_right = 600

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to capture video from camera.")
        break

    frame_copy = frame.copy()

    # Define and extract the Region of Interest (ROI) from the frame
    # Ensuring the ROI does not exceed image dimensions
    roi = frame_copy[max(0, roi_top):min(frame.shape[0], roi_bottom), max(0, roi_left):min(frame.shape[1], roi_right)]

    # Convert the ROI to grayscale and apply Gaussian blur
    gray = cv2.GaussianBlur(cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY), (7, 7), 0)

    if num_frames < 60:
        background = calc_accum_avg(gray, accumulated_weight, background)
        display_text = 'WAIT. GETTING BACKGROUND' if num_frames < 59 else ''
    else:
        hand = segment(gray, background, 25)
        if hand:
            thresholded, hand_segment = hand
            
            # Adjust the contours for correct visualization
            shifted_contour = [cnt + np.array([roi_left, roi_top], dtype=np.int32) for cnt in hand_segment]
            cv2.drawContours(frame_copy, shifted_contour, -1, (255, 0, 0), 1)
            
            # Count the fingers on the segmented hand
            fingers = count_fingers(thresholded, hand_segment)
            display_text = f'{fingers} fingers'
            cv2.imshow('Thresholded', thresholded)

    if display_text:
        text_position = (roi_left + 70, roi_top + 50)
        cv2.putText(frame_copy, display_text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
    cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0, 0, 255), 5)
    cv2.imshow('Finger Count', frame_copy)
    
    num_frames += 1

    if cv2.waitKey(1) & 0xFF == 27: # Press ESC to exit
        break

cam.release()
cv2.destroyAllWindows()